# Data Processing: Document Conversion with Standard Docling

This notebook uses **standard (non-VLM)** [Docling](https://docling-project.github.io/docling/) techniques to convert PDF documents into markdown and the [Docling Document](https://docling-project.github.io/docling/concepts/docling_document/) format, a structured representation of the original document that can be exported as JSON.

Conversions using the standard pipeline options generally yield good and fast results for most documents. In some cases, however, alternative conversion pipelines can lead to better outcomes. For instance, forcing OCR is effective for scanned documents or images that contain text to be extracted and analyzed. In cases where relevant information is contained within formulas, code, or pictures; enrichments might be useful. All these use cases are supported by this notebook.

## 📦 Installation

Install the [Docling](https://docling-project.github.io/docling/) package into this notebook environment. Run this once per session, it may take a minute. If you restart the kernel or change runtimes, re-run this cell before continuing.

In [ ]:
!pip install -qq docling

## 🔧 Configuration

### Set files to convert

Set the list of PDF files to convert. You can mix public web URLs and local file paths, each entry will be processed in order. Replace the examples with your own documents as needed.

In [ ]:
files = [
    "https://raw.githubusercontent.com/py-pdf/sample-files/refs/heads/main/001-trivial/minimal-document.pdf",
    "https://github.com/docling-project/docling/raw/v2.43.0/tests/data/pdf/2203.01017v2.pdf"
]

### Set output directory

Choose where to save results. This notebook creates the folder if it doesn’t exist and writes one `json` ([Docling Document](https://docling-project.github.io/docling/concepts/docling_document/)) and one `md` file per source file, using the source's base name.

In [ ]:
from pathlib import Path

output_dir_name = "document-conversion-standard/output"

output_dir = Path(output_dir_name)
output_dir.mkdir(parents=True, exist_ok=True)

### Configure conversion pipelines

Next we create the configuration options for the conversion pipelines supported by this notebook. 

The next cell configures two combinations of pipeline options: the **default (standard)** options, and a variant that **forces OCR** on the entire document. In a later cell you'll choose either the `standard` or `ocr` pipeline options depending on which conversion technique you'd like to use.

Note: OCR requires the Tesseract binary to run. Please refer to the Docling [installation](https://docling-project.github.io/docling/installation/) docs if you're not running this notebook from a Workbench image that has it installed already. 

For additional customization and a complete reference of Docling's conversion pipeline configuration, check the [official documentation](https://docling-project.github.io/docling/examples/).

In [ ]:
from docling.datamodel.accelerator_options import AcceleratorDevice, AcceleratorOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    TesseractOcrOptions,
    PdfPipelineOptions,
)
from docling.backend.docling_parse_v4_backend import DoclingParseV4DocumentBackend

def create_standard_options() -> PdfPipelineOptions:
    """Create base pipeline options with standard settings."""
    pipeline_options = PdfPipelineOptions()
    pipeline_options.generate_picture_images = True
    pipeline_options.do_table_structure = True
    pipeline_options.table_structure_options.do_cell_matching = True
    pipeline_options.accelerator_options = AcceleratorOptions(
        num_threads=4, device=AcceleratorDevice.AUTO
    )
    return pipeline_options

# Standard converter
standard_options = create_standard_options()

# OCR converter: force OCR on the entire page
%env TESSDATA_PREFIX=/usr/share/tesseract/tessdata
ocr_options = create_standard_options()
ocr_options.do_ocr = True
ocr_options.ocr_options = TesseractOcrOptions(force_full_page_ocr=True)

def get_pipeline_options(pipeline_options_name: str = "standard") -> PdfPipelineOptions:
    """Get the configured pipeline options based on name.

    Args:
        pipeline_options_name: One of "standard" or "ocr"
    
    Returns:
        PdfPipelineOptions instance
        
    Raises:
        ValueError: If pipeline_options_name is not recognized
    """
    pipeline_options = {
        "standard": standard_options,
        "ocr": ocr_options
    }

    if pipeline_options_name not in pipeline_options:
        raise ValueError(
            f"Unknown pipeline options name: '{pipeline_options_name}'. "
            f"Choose from {list(pipeline_options.keys())}"
        )
    
    return pipeline_options[pipeline_options_name]

### Choose a conversion pipeline

Next we choose the conversion pipeline to be used in the conversion.

The **standard** pipeline generally yield good and fast results for the majority of documents. However, if you didn't get good results with the standard pipeline, or if you're converting scanned documents or ones that contain relevant text information within images, consider using the **ocr** pipeline. Just set `pipeline_to_use` to either `standard` or `ocr` accordingly.

In [ ]:
# Set the pipeline to use (either "standard" or "ocr")
pipeline_to_use = "standard"

pipeline_options = get_pipeline_options(pipeline_to_use)

print(f"✓ Using '{pipeline_to_use}' pipeline")

### Configure enrichments

Depending on the characteristics of the documents being converted, you may benefit from the use of enrichments.

Docling supports the enrichment of conversion pipelines with additional steps that'll process specific document components like code blocks, formulas, and pictures.

All enrichment features are disabled by default, but you may enable them individually by setting one or more of the enrichment options in the next cell to `True`. Note that the extra steps usually require extra models executions, which may increase the processing time consistently.

 * `do_code_enrichment`: Performs advanced parsing of code blocks and sets the code language of each block accordingly.
 * `do_formula_enrichment`: Analyzes equations and extracts their LaTeX representation.
 * `do_picture_description`: Generates captions for pictures with a vision model.
 * `do_picture_classification`: Classifies pictures, for example chart types, flow diagrams, logos, or signatures.
  
For additional customization and a complete reference of Docling's enrichments, check the [official documentation](https://docling-project.github.io/docling/usage/enrichments/).

In [ ]:
# Sets code and formula enrichments and picture description and classification
pipeline_options.do_code_enrichment = False
pipeline_options.do_formula_enrichment = False
pipeline_options.do_picture_description = False
pipeline_options.do_picture_classification = False

# If you enable enrichments, you may benefit from increasing the image scale (e.g. to 2)
pipeline_options.images_scale = 1


## ✨ Conversion

Finally, use the pipeline options we configured to convert every document into one `json` ([Docling Document](https://docling-project.github.io/docling/concepts/docling_document/)) and one `md` (markdown), which will be stored in the output directory configured earlier.

In [ ]:
import json
from docling_core.types.doc import ImageRefMode

# Create the document converter
converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options,
                backend=DoclingParseV4DocumentBackend,
            )
        }
    )

confidence_reports = {}

if not files:
    raise ValueError("No input files specified. Please set the 'files' list above.")

for file in files:
    # Convert the file
    print(f"Converting {file}...")

    result = converter.convert(file)
    document = result.document
    dictionary = document.export_to_dict()

    # Calculate conversion confidence
    confidence_reports[file] = result.confidence

    file_path = Path(file)

    # Export the document to JSON
    json_output_path = (output_dir / f"{file_path.stem}.json")
    with open(json_output_path, "w", encoding="utf-8") as f:
        json.dump(dictionary, f)
        print(f"✓ Path of JSON output is: {json_output_path.resolve()}")

    # Export the document to Markdown
    md_output_path = output_dir / f"{file_path.stem}.md"
    with open(md_output_path, "w", encoding="utf-8") as f:
        markdown = document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED)
        f.write(markdown)
        print(f"✓ Path of markdown output is: {md_output_path.resolve()}")

### Conversion confidence

When converting a document, Docling can calculate how confident it is in the quality of the conversion. This *confidence* is expressed as both a *score* and a *grade*. The score is a numeric value between 0 and 1, and the grade is a label that can be **poor**, **fair**, **good**, or **excellent**. If Docling is unable to calculate a confidence grade, the value will be marked as *unspecified*.

If your document receives a low score (for example, below 0.8) and a grade of *poor* or *fair*, you'll probably benefit from using a different conversion technique. In that case, go back to the *Conversion* section and try selecting a different approach (e.g. forcing OCR) and compare the results.

In [ ]:
for file, confidence_report in confidence_reports.items():
    print(f"✓ Conversion confidence for {file}:")
    
    print(f"Average confidence: \x1b[1m{confidence_report.mean_grade.name}\033[0m (score {confidence_report.mean_score:.3f})")
    
    low_score_pages = []
    for page in confidence_report.pages:
        page_confidence_report = confidence_report.pages[page]
        if page_confidence_report.mean_score < confidence_report.mean_score:
            low_score_pages.append(page)

    print(f"Pages that scored lower than average: {', '.join(str(x + 1) for x in low_score_pages) or 'none'}")
    
    print()

## 🍩 Additional resources

For additional example notebooks related to Data Processing, check the [Open Data Hub Data Processing](https://github.com/opendatahub-io/odh-data-processing/) repository on GitHub.

### Any Feedback?

We'd love to hear if you have any feedback on this or any other notebook in this series! Please [open an issue](https://github.com/opendatahub-io/odh-data-processing/issues) and help us improve our demos.